# Standard evaluators and target functions.
In this notebook we will demonstrate how to use the target functions with the standard evaluators.

In [1]:
import inspect
import json
import pandas as pd
import os

from pprint import pprint
from promptflow.core import AzureOpenAIModelConfiguration
from promptflow.evals.evaluate import evaluate
from promptflow.evals.evaluators import F1ScoreEvaluator, RelevanceEvaluator

## Target function
We will use a simple `Ask Wiki` application to get answers to questions from wikipedia. We will use `evaluate` API to evaluate `Ask Wiki` applicaton

`Ask Wiki` needs following environment variables to be set

- AZURE_OPENAI_API_KEY
- AZURE_OPENAI_API_VERSION
- AZURE_OPENAI_DEPLOYMENT
- AZURE_OPENAI_ENDPOINT

In [2]:
# Use the following code to set the environment variables if not already set. If set, you can skip this step.

os.environ["AZURE_OPENAI_API_KEY"] = "<api_key>"
os.environ["AZURE_OPENAI_API_VERSION"] = "<api_version>"
os.environ["AZURE_OPENAI_DEPLOYMENT"] = "<deployment>"
os.environ["AZURE_OPENAI_ENDPOINT"] = "<endpoint>"

In [3]:
from askwiki.askwiki import ask_wiki

ask_wiki("What is the capital of India?")


{'answer': 'The capital of India is New Delhi.',
 'context': 'Content: Capital punishment in India is a legal penalty for some crimes under the country\'s main substantive penal legislation, the Indian Penal Code, as well as other laws. Executions are carried out by hanging as the primary method of execution per Section 354(5) of the Criminal Code of Procedure, 1973 is "Hanging by the neck until dead", and is imposed only in the \'rarest of cases\'.[1][2]. Currently, there are around 539 [3] prisoners on death row in India. The most recent executions in India took place in March 2020, when four of the 2012 Delhi gang rape and murder perpetrators were executed at the Tihar Jail in Delhi.[4]. In the Code of Criminal Procedure (CrPC), 1898 death was the default punishment for murder and required the concerned judges to give reasons in their judgment if they wanted to give life imprisonment instead.[5] By an amendment to the CrPC in 1955, the requirement of written reasons for not imposing

## Data
Reading existing dataset which has bunch of questions we can Ask Wiki

In [4]:
df = pd.read_json('data/data.jsonl', lines=True)
print(df.head())

                                      question   ground_truth
0               When was United Stated found ?           1776
1               What is the capital of France?          Paris
2  Who is the best tennis player of all time ?  Roger Federer


## Configuration
To use Relevance Evaluator, we will Azure Open AI model details that can be passed as model config.


In [5]:
configuration = AzureOpenAIModelConfiguration(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"] ,
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT"],
)

## Running Relevance Evaluator to understand its input and output

In [6]:
relevance_evaluator = RelevanceEvaluator(model_config=configuration)

relevance_evaluator(question="What is the capital of India?", answer="New Delhi is Capital of India" ,context="India is a country in South Asia.")

[2024-05-01 11:58:13,006][flowinvoker][INFO] - Getting connections from pf client with provider from args: local...
[2024-05-01 11:58:13,013][flowinvoker][INFO] - Promptflow get connections successfully. keys: dict_keys(['connection_5259131854148614271'])
[2024-05-01 11:58:13,014][flowinvoker][INFO] - Promptflow executor starts initializing...
[2024-05-01 11:58:13,328][flowinvoker][INFO] - Promptflow executor initiated successfully.
[2024-05-01 11:58:13,330][flowinvoker][INFO] - Validating flow input with data {'question': 'What is the capital of India?', 'answer': 'New Delhi is Capital of India', 'context': 'India is a country in South Asia.'}
[2024-05-01 11:58:13,331][flowinvoker][INFO] - Execute flow with data {'question': 'What is the capital of India?', 'answer': 'New Delhi is Capital of India', 'context': 'India is a country in South Asia.'}


2024-05-01 11:58:13 -0700   31040 execution.flow     INFO     Start executing nodes in thread pool mode.
2024-05-01 11:58:13 -0700   31040 execution.flow     INFO     Start to run 3 nodes with concurrency level 16.
2024-05-01 11:58:13 -0700   31040 execution.flow     INFO     Executing node validate_inputs. node run id: 3c31376b-a5c6-43c5-86bb-9b99ca67f124_validate_inputs_00755a06-7129-41c4-8053-41724904f606
2024-05-01 11:58:13 -0700   31040 execution.flow     INFO     Node validate_inputs completes.
2024-05-01 11:58:13 -0700   31040 execution.flow     INFO     The node 'query_llm' will be executed because the activate condition is met, i.e. '${validate_inputs.output}' is equal to 'True'.
2024-05-01 11:58:13 -0700   31040 execution.flow     INFO     Executing node query_llm. node run id: 3c31376b-a5c6-43c5-86bb-9b99ca67f124_query_llm_7d9feebf-70c2-4d6e-9184-45b2a577f371
2024-05-01 11:58:16 -0700   31040 execution.flow     INFO     Node query_llm completes.
2024-05-01 11:58:16 -0700   3

{'gpt_relevance': 5.0}

## Running custom evaluator **Perplexity**

In [7]:
# Note: The following evaluator is a defined in directory evaluators/blocklist

from evaluators.blocklist.blocklist import BlocklistEvaluator

blocklist_evaluator = BlocklistEvaluator(blocklist="bad, worst, terrible")

blocklist_evaluator(answer="New Delhi is Capital of India")

{'score': True}

## Run the evaluation

In [8]:
results = evaluate(
    data="data/data.jsonl",
    target=ask_wiki,
    evaluators={
        "blocklist": blocklist_evaluator,
        "relevance": relevance_evaluator,
    },
)

[2024-05-01 11:58:16,553][promptflow][WARNING] - Code path is not specified, use current working directory: c:/Users/anksing/AppData/Roaming/JetBrains/PyCharmCE2024.1/scratches/evaluate-target


Prompt flow service has started...


[2024-05-01 11:58:21,004][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run evaluate_target_variant_0_20240501_115816_568127, log path: C:\Users\anksing\.promptflow\.runs\evaluate_target_variant_0_20240501_115816_568127\logs.txt


You can view the traces in local from http://localhost:23333/v1.0/ui/traces/?#run=evaluate_target_variant_0_20240501_115816_568127
2024-05-01 11:58:21 -0700   31040 execution          WARNING  Starting run without column mapping may lead to unexpected results. Please consult the following documentation for more information: https://aka.ms/pf/column-mapping
2024-05-01 11:58:21 -0700   31040 execution.bulk     INFO     Current system's available memory is 31773.09375MB, memory consumption of current process is 336.4375MB, estimated available worker count is 31773.09375/336.4375 = 94
2024-05-01 11:58:21 -0700   31040 execution.bulk     INFO     Set process count to 3 by taking the minimum value among the factors of {'default_worker_count': 4, 'row_count': 3, 'estimated_worker_count_based_on_memory_usage': 94}.
2024-05-01 11:58:23 -0700   31040 execution.bulk     INFO     Process name(SpawnProcess-2)-Process id(41492)-Line number(0) start execution.
2024-05-01 11:58:23 -0700   31040 execut

[2024-05-01 11:58:43,284][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run evaluate_target_variant_0_20240501_115839_112646, log path: C:\Users\anksing\.promptflow\.runs\evaluate_target_variant_0_20240501_115839_112646\logs.txt


You can view the traces in local from http://localhost:23333/v1.0/ui/traces/?#run=evaluate_target_variant_0_20240501_115839_112646
2024-05-01 11:58:43 -0700   31040 execution          WARNING  Starting run without column mapping may lead to unexpected results. Please consult the following documentation for more information: https://aka.ms/pf/column-mapping
2024-05-01 11:58:43 -0700   31040 execution.bulk     INFO     Current system's available memory is 31774.84375MB, memory consumption of current process is 340.53125MB, estimated available worker count is 31774.84375/340.53125 = 93
2024-05-01 11:58:43 -0700   31040 execution.bulk     INFO     Set process count to 3 by taking the minimum value among the factors of {'default_worker_count': 4, 'row_count': 3, 'estimated_worker_count_based_on_memory_usage': 93}.
2024-05-01 11:58:46 -0700   31040 execution.bulk     INFO     Process name(SpawnProcess-8)-Process id(13812)-Line number(0) start execution.
2024-05-01 11:58:46 -0700   31040 exec

[2024-05-01 11:58:58,518][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run evaluate_target_variant_0_20240501_115854_376324, log path: C:\Users\anksing\.promptflow\.runs\evaluate_target_variant_0_20240501_115854_376324\logs.txt


You can view the traces in local from http://localhost:23333/v1.0/ui/traces/?#run=evaluate_target_variant_0_20240501_115854_376324
2024-05-01 11:58:58 -0700   31040 execution          WARNING  Starting run without column mapping may lead to unexpected results. Please consult the following documentation for more information: https://aka.ms/pf/column-mapping
2024-05-01 11:58:58 -0700   31040 execution.bulk     INFO     Current system's available memory is 31755.29296875MB, memory consumption of current process is 340.71484375MB, estimated available worker count is 31755.29296875/340.71484375 = 93
2024-05-01 11:58:58 -0700   31040 execution.bulk     INFO     Set process count to 3 by taking the minimum value among the factors of {'default_worker_count': 4, 'row_count': 3, 'estimated_worker_count_based_on_memory_usage': 93}.
2024-05-01 11:59:01 -0700   31040 execution.bulk     INFO     Process name(SpawnProcess-14)-Process id(43812)-Line number(0) start execution.
2024-05-01 11:59:01 -0700

View the results

In [9]:
pprint(results)

{'metrics': {'blocklist.score': 1.0, 'relevance.gpt_relevance': 5.0},
 'rows': [{'inputs.ground_truth': '1776',
           'inputs.question': 'When was United Stated found ?',
           'outputs.answer': 'The United States was established after the War '
                             'of Independence from Great Britain, which '
                             'happened in the late 18th century. However, the '
                             'exact date is often considered to be July 4, '
                             '1776, when the Declaration of Independence was '
                             'signed.',
           'outputs.blocklist.score': True,
           'outputs.context': 'Content: The Founding Fathers of the United '
                              'States, commonly referred to as the Founding '
                              'Fathers, were a group of late-18th-century '
                              'American revolutionary leaders who united the '
                              'Thirteen 

In [10]:
pd.DataFrame(results["rows"])

,outputs.answer,outputs.context,inputs.question,inputs.ground_truth,outputs.blocklist.score,outputs.relevance.gpt_relevance
0,The United States was established after the Wa...,Content: The Founding Fathers of the United St...,When was United Stated found ?,1776,True,5
1,The document doesn't provide specific informat...,Content: A closed-ended question refers to any...,What is the capital of France?,Paris,True,5
2,The documents provided do not contain informat...,Content: This article covers the period from 1...,Who is the best tennis player of all time ?,Roger Federer,True,5
